In [1]:
# Import necessary packages to handle DataFrames, JSON files, and create NaN values (pandas, json, and numpy)
import pandas as pd
import json
#“Normalize” semi-structured JSON data into a flat table
from pandas.io.json import json_normalize

# Set iPython's max column width to 1000
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', -1)

#import numpy
import numpy as np
#import matplotlib
import matplotlib.pyplot as plt
#importing regular expression 
import re
#remove warnings
#import warnings
#warnings.filterwarnings('ignore')


In [136]:
# Loading the dataframe into df
#with open('disresort1.json', 'r', encoding="utf-8") as json_file:
    #json_work = json.load(json_file)

#df = pd.io.json.json_normalize(json_work)
df = pd.read_json('disresort1.json',lines = True)
#json_normalize(df)
df.head()

def flatten_json(nested_json: dict, exclude: list=['']) -> dict:
    """
    Flatten a list of nested dicts.
    """
    out = dict()
    def flatten(x: (list, dict, str), name: str='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude:
                    flatten(x[a], f'{name}{a}_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, f'{name}{i}_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out
flatten_df=flatten_json(df)
#print(flatten_df)
type(flatten_df)
flatten_df1 = pd.DataFrame([flatten_df], columns=flatten_df.keys())
flatten_df1.columns
# View the head of df 
#df.head()

Index([''], dtype='object')

In [132]:
#look at the columns we can use
print(df.columns)

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'reply_count', 'retweet_count', 'retweeted', 'retweeted_status',
       'source', 'text', 'timestamp_ms', 'truncated', 'user'],
      dtype='object')


In [9]:
#see if the retweeted colum is useful or not. 
print(df.retweeted.value_counts())
#look into RT and retweet_status might have to run a regular expression. 
#df.retweeted_status.value_counts()
print(df.retweeted_status.isna().sum())
print(df.shape)
#444 original tweets out of the 1118 sample size we took 


False    1118
Name: retweeted, dtype: int64
444
(1118, 36)


In [10]:
#take columns for the sentiment analysis 
df1 = df[['text','id',
          'lang','created_at',
          'user','source','retweeted_status',
          'extended_tweet', 'extended_entities',
         'timestamp_ms', 'entities']]

#creating a text count column
df1.assign(text_length=df1['text'].apply(len))
print(df1.shape)

(1118, 11)


In [11]:
#lets look at our new selected columns 
print(df1.head(5))

                                                                                                                                           text  \
0  RT @tdrblognote: これが新しいミッキー花壇ですか～ 分離型です。イベントフォトロケを作るなら、ワールドバザール側ですかね。ミッキー側はフォトロケ置けるスペースが無い。 #TDR_now https://t.co/xYEQmaDLHm                   
1  @OrangeGrove55 Galaxy’s Edge at Walt Disney World isn’t a failure, but like Disneyland it’s underperforming. But ho… https://t.co/3KdOXXhmNV   
2  RT @hp_yec: 昨日となんか違うと思ったら\nミッキー花壇帰ってました！\nお帰り！\n\n#TDR_now https://t.co/KUib5PtJlT                                                             
3  RT @gourmetdyy: New #Halloween themed cups! #ShanghaiDisneyland  #SHDL #上海ディズニーランド #Disney #Disneypark #Disneyland  #Duffy https://t.co/2fa…   
4  Why would Disney need to promote #StarWars: #GalaxysEdge so much if everything is hunky-dory?                                                  

                    id lang          created_at  \
0  1176336509575712768  ja  2019-09-24 03:24:13   
1  117633651996

In [12]:
#Created_at and timestamp_ms is a repeat need to drop the column.
del df1['timestamp_ms']
df1.shape

(1118, 10)

#lets look at some of the nested dictionaries 
#entites
#extended_entities 
#extended_tweet
#retweeted_status
#user

In [13]:
#look at entities
#https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/entities-object
#taking the keys and values from df1.entities; Return the dictionary's key-value pairs:
entities = df1['entities'].items()
#turning into a list 
entities = list(zip(entities))
#convert list to an array 
entities = np.asarray(entities)
#lets go up by 100 to see what nested dictionaries are important 
print(entities[0::100])

[[[0
   {'hashtags': [{'text': 'TDR_now', 'indices': [92, 100]}], 'urls': [], 'user_mentions': [{'screen_name': 'tdrblognote', 'name': 'ＫｏＺ', 'id': 109227124, 'id_str': '109227124', 'indices': [3, 15]}], 'symbols': [], 'media': [{'id': 1176330997974982657, 'id_str': '1176330997974982657', 'indices': [101, 124], 'media_url': 'http://pbs.twimg.com/media/EFMqz2iUYAEmQUQ.jpg', 'media_url_https': 'https://pbs.twimg.com/media/EFMqz2iUYAEmQUQ.jpg', 'url': 'https://t.co/xYEQmaDLHm', 'display_url': 'pic.twitter.com/xYEQmaDLHm', 'expanded_url': 'https://twitter.com/tdrblognote/status/1176331009068896256/photo/1', 'type': 'photo', 'sizes': {'small': {'w': 480, 'h': 640, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w': 480, 'h': 640, 'resize': 'fit'}, 'large': {'w': 480, 'h': 640, 'resize': 'fit'}}, 'source_status_id': 1176331009068896256, 'source_status_id_str': '1176331009068896256', 'source_user_id': 109227124, 'source_user_id_str': '109227124'}]}]]

 [[100
   

In [ ]:
#hashtags with a nested dictionary text,
#user_mentions with screen_name, name and id
#Extended_tweets might be better to use since it contains more of the information. 

In [14]:
for k,v in entities[800]:
    if pd.isnull(v):
        print('nan value')
    else:
        for k1,v1 in v.items():
            print(k1,v1)

hashtags [{'text': 'refusetohavefun', 'indices': [95, 111]}]
urls [{'url': 'https://t.co/THWvRommAm', 'expanded_url': 'https://twitter.com/i/web/status/1176351457160753152', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [113, 136]}]
user_mentions []
symbols []


Entities provide metadata and additional contextual information about content posted on Twitter. The entities section provides arrays of common things included in Tweets: hashtags, user mentions, links, stock tickers (symbols), Twitter polls, and attached media. These arrays are convenient for developers when ingesting Tweets, since Twitter has essentially pre-processed, or pre-parsed, the text body. Instead of needing to explicitly search and find these entities in the Tweet body, your parser can go straight to this JSON section and there they are.
Beyond providing parsing conveniences, the entities section also provides useful ‘value-add’ metadata. For example, if you are using the Enhanced URLs enrichment, URL metadata include fully-expanded URLs, as well as associated website titles and descriptions. Another example is when there are user mentions, the entities metadata include the numeric user ID, which are useful when making requests to many Twitter APIs.
Every Tweet JSON payload includes an entities section, with the minimum set of hashtags, urls, user_mentions, and symbols attributes, even if none of those entities are part of the Tweet message. For example, if you examine the JSON for a Tweet with a body of “Hello World!” and no attached media, the Tweet’s JSON will include the following content with entity arrays containing zero items:

In [15]:
#pull out entities 
df2 = df1

#print(df2[''])
# foo = pd.DataFrame(df2['entities'][0]['hashtags'])
# foo.iloc[:,:]
df2['entities_hashtag'] = np.array([x['hashtags'] for x in df2['entities']])
df2['entities_user_mentions'] = np.array([x['user_mentions'] for x in df2['entities']])
print(df2['entities_hashtag'][0:3])
print(df2['entities_user_mentions'][0:3])



   

0    [{'text': 'TDR_now', 'indices': [92, 100]}]
1    []                                         
2    [{'text': 'TDR_now', 'indices': [46, 54]}] 
Name: entities_hashtag, dtype: object
0    [{'screen_name': 'tdrblognote', 'name': 'ＫｏＺ', 'id': 109227124, 'id_str': '109227124', 'indices': [3, 15]}]                              
1    [{'screen_name': 'OrangeGrove55', 'name': 'OrangeGrove55', 'id': 850410392790614017, 'id_str': '850410392790614017', 'indices': [0, 14]}]
2    [{'screen_name': 'hp_yec', 'name': 'しゅんD垢９月23.29', 'id': 731025834601930752, 'id_str': '731025834601930752', 'indices': [3, 10]}]        
Name: entities_user_mentions, dtype: object


/Users/justin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/justin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
#look into retweeted_status
#https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object

#taking the keys and values from df1 
rt_stat = df1['retweeted_status'].items()
#turning into a list 
rt_stat = list(zip(rt_stat))
#convert list to an array 
rt_stat = np.asarray(rt_stat)
#lets go up by 100 to see what nested dictionaries are important 
print(rt_stat[0::100])

[[[0
   {'created_at': 'Tue Sep 24 03:02:22 +0000 2019', 'id': 1176331009068896256, 'id_str': '1176331009068896256', 'text': 'これが新しいミッキー花壇ですか～ 分離型です。イベントフォトロケを作るなら、ワールドバザール側ですかね。ミッキー側はフォトロケ置けるスペースが無い。 #TDR_now https://t.co/xYEQmaDLHm', 'display_text_range': [0, 83], 'source': '<a href="https://github.com/TwidereProject" rel="nofollow">Twidere for Android #8</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 109227124, 'id_str': '109227124', 'name': 'ＫｏＺ', 'screen_name': 'tdrblognote', 'location': '週末はほぼ舞浜', 'url': 'http://tdrblognote.blogspot.jp/', 'description': 'LUMIX DC-GH5S, LUMIX DC-GH5, LUMIX DC-GX7 MarkⅢ, LUMIX DC-FT7, SONY Xperia XZ Premium SO-04J, dji OSMO pocket  フォロワーの方で、長期間ツイートされてない方、鍵付でプロフの適当な方は不定期にブロックいたします。', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 2131, 'friends_count': 99, 'li

In [17]:
#checking the keys for retweeted_status 
for k,v in rt_stat[0]:
    if pd.isnull(v):
        print('nan value')
    else:
        for k1, v1 in v.items():
            print(k1)

created_at
id
id_str
text
display_text_range
source
truncated
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
in_reply_to_screen_name
user
geo
coordinates
place
contributors
is_quote_status
quote_count
reply_count
retweet_count
favorite_count
entities
extended_entities
favorited
retweeted
possibly_sensitive
filter_level
lang


In [110]:
#[unicode(x.strip()) if x is not None else '' for x in row]
#([k for k, v in tags] if tags is not None else [])
# df2['rt_stat_id'] = np.array([x['id'] for x in df2['retweeted_status']])
#df2['rt_stat_text'] = np.array([x['text'] for x in df2['retweeted_status']])
# df2['rt_stat_source'] = np.array([x['source'] for x in df2['retweeted_status']])
# df2['rt_stat_user'] = np.array([x['user'] for x in df2['retweeted_status']])
# df2['rt_stat_retweet_count'] = np.array([x['retweet_count'] for x in df2['retweeted_status']])
# df2['rt_stat_extended_entities'] = np.array([x['extended_entities'] for x in df2['retweeted_status']])
# df2['rt_stat_entities'] = np.array([x['entities'] for x in df2['retweeted_status']])
# df2['rt_stat_favorited'] = np.array([x['favorited'] for x in df2['retweeted_status']])
# df2['rt_stat_lang'] = np.array([x['lang'] for x in df2['retweeted_status']])
#drop entities 
#df2.drop('entities', axis=1, inplace=True)
df2.columns
df2.head()

#foo = pd.DataFrame(df2['retweeted_status'])
df2['retweeted_status'] = df2['retweeted_status'].replace(np.nan, 0)
df2.retweeted_status[6]
#df2['rt_created_at'] = np.array([x['created_at'] for x in df2['retweeted_status'] if not 0])
rt = df2.loc[:,'retweeted_status']

rt_s = list(rt_stat)
type(rt)

#df2.loc[0:3,'retweeted_status']

pandas.core.series.Series

In [ ]:
#look at extended_tweet
#https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/extended-entities-object

#taking the keys and values from df1 
ex_tweet = df1['extended_tweet'].items()
#turning into a list 
ex_tweet  = list(zip(ex_tweet))
#convert list to an array 
ex_tweet  = np.asarray(ex_tweet)
#lets go up by 100 to see what nested dictionaries are important 
print(ex_tweet[0::100])

In [ ]:
for k,v in ex_tweet[800]:
    if pd.isnull(v):
        print('nan value')
    else:
        for k1, v1 in v.items():
            print(k1,v1)

In [ ]:
#full_text full text that is not truncated
#display_text_range not useful
#entities
#extended_entities looks at the image and photos won't need in our anaylsis 

Extended tweets has entities and extended entities nested within it as a repeat. 
Would want to collect the full_text information 
Text range not necessary. 
Entities we would want to collect hashtags and text area and not the indicies. 
user mentions

http://docs.tweepy.org/en/latest/extended_tweets.html




In [ ]:
#look at user
#https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object
#taking the keys and values from df1 
user = df1['user'].items()
#turning into a list 
user = list(zip(user))
#convert list to an array 
user  = np.asarray(user)
#lets go up by 100 to see what nested dictionaries are important 
print(user[0::100])

In [ ]:
for k,v in user[800]:
    if pd.isnull(v):
        print('nan value')
    else:
        for k1, v1 in v.items():
            print(k1)

In [ ]:
#look at extended_entities
#https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/extended-entities-object
ex_ent = df1['extended_entities'].items()
#turning into a list 
ex_ent = list(zip(ex_ent))
#convert list to an array 
ex_ent  = np.asarray(ex_ent)
#lets go up by 100 to see what nested dictionaries are important 
print(ex_ent[0::100])

In [ ]:
for k,v in ex_ent[600]:
    if pd.isnull(v):
        print('nan value')
    else:
        for k1, v1 in v.items():
            print(k1)

All Tweets with attached photos, videos and animated GIFs will include an extended_entities JSON object. The extended_entities object contains a single media array of media objects (see the entities section for its data dictionary). No other entity types, such as hashtags and links, are included in the extended_entities section. The media object in the extended_entities section is identical in structure to the one included in the entities section.
Tweets can only have one type of media attached to it. For photos, up to four photos can be attached. For videos and GIFs, one can be attached. Since the media type metadata in the extended_entities section correctly indicates the media type (‘photo’, ‘video’ or ‘animated_gif’), and supports up to 4 photos, it is the preferred metadata source for native media.

Including Media wouldn't be as useful in the analysis.  

In [26]:
#lets remove the extended entities column 
del df2['extended_entities']

In [20]:
#selecting dictionaries from user, nested dictionary

df2['user_id'] = np.array([x['id'] for x in df2['user']])
df2['user_screen_name'] = np.array([x['screen_name'] for x in df2['user']])
df2['user_location'] = np.array([x['location'] for x in df2['user']])
df2['user_descrip'] = np.array([x['description'] for x in df2['user']])
df2['user_fol_count'] = np.array([x['followers_count'] for x in df2['user']])
df2['user_fr_count'] = np.array([x['friends_count'] for x in df2['user']])
df2['user_fav_count'] = np.array([x['favourites_count'] for x in df2['user']])
df2['user_status_count'] = np.array([x['statuses_count'] for x in df2['user']])
df2['user_created_at'] = np.array([x['created_at'] for x in df2['user']])
df2['user_listed_count'] = np.array([x['listed_count'] for x in df2['user']])

#drop users column
df2=df2.drop(columns='user')
df2.head()

/Users/justin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/justin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/justin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

text  \
0  RT @tdrblognote: これが新しいミッキー花壇ですか～ 分離型です。イベントフォトロケを作るなら、ワールドバザール側ですかね。ミッキー側はフォトロケ置けるスペースが無い。 #TDR_now https://t.co/xYEQmaDLHm                   
1  @OrangeGrove55 Galaxy’s Edge at Walt Disney World isn’t a failure, but like Disneyland it’s underperforming. But ho… https://t.co/3KdOXXhmNV   
2  RT @hp_yec: 昨日となんか違うと思ったら\nミッキー花壇帰ってました！\nお帰り！\n\n#TDR_now https://t.co/KUib5PtJlT                                                             
3  RT @gourmetdyy: New #Halloween themed cups! #ShanghaiDisneyland  #SHDL #上海ディズニーランド #Disney #Disneypark #Disneyland  #Duffy https://t.co/2fa…   
4  Why would Disney need to promote #StarWars: #GalaxysEdge so much if everything is hunky-dory?                                                  

                    id lang          created_at              source  \
0  1176336509575712768  ja  2019-09-24 03:24:13  Twitter for iPhone   
1  1176336519969345536  en  2019-09-24 03:24:16  Twitter for iPhone   
2  1176336531339919360  ja  2019-09-24 03:24:19  Twitter for iPhone   
3  1176336557969567744  en  2019-09-24 03:24:25  Twitter for iPhone   
4  1176336572305874944  en  2019-09-24 03:24:28  Twitter Web App      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [21]:
# #selecting dictionaries from retweeted_status, nested dictionary
# df1['retweeted_status_screen_name'] = df1['retweeted_status'].transform(lambda x: x['screen_name'])
# df1['retweeted_status_location'] = df1['retweeted_status'].transform(lambda x: x['location'])
# df1['retweeted_status_description'] = df1['retweeted_status'].transform(lambda x: x['description'])
# df1['retweeted_status_followers_count'] = df1['retweeted_status'].transform(lambda x: x['followers_count'])
# df1['retweeted_status_friends_count'] = df1['retweeted_status'].transform(lambda x: x['friends_count'])
# df1['retweeted_status_favourites_count'] = df1['retweeted_status'].transform(lambda x: x['favourites_count'])
# df1['retweeted_status_statuses_count'] = df1['retweeted_status'].transform(lambda x: x['statuses_count'])
# df1['retweeted_status_created_at'] = df1['retweeted_status'].transform(lambda x: x['created_at'])
# df1['retweeted_status_listed_count'] = df1['retweeted_status'].transform(lambda x: x['listed_count'])
# df1['retweeted_status_in_reply_to_screen_name'] = df1['retweeted_status'].transform(lambda x: x['in_reply_to_screen_name'])
# df1['retweeted_status_source'] = df1['retweeted_status'].transform(lambda x: x['source'])
# df1['retweeted_status_retweet_count'] = df1['retweeted_status'].transform(lambda x: x['retweet_count'])
# df1['retweeted_status_retweeted'] = df1['retweeted_status'].transform(lambda x: x['retweeted'])
                                          
# #drop retweeted_status column
# df1=df1.drop(columns='retweeted_status')

In [ ]:
#clean up extended entities

In [ ]:
#clean up extended_tweets

In [ ]:
#clean up entities

In [ ]:
#Regular expression to get the RT out

In [ ]:
#Regular expression to get the @ out

In [25]:
#Clean up the source 
df2.source[0]
#print(df2.source.unique())
from bs4 import BeautifulSoup
df2['source'] = [BeautifulSoup(text).get_text() for text in df2['source'] ]
print(df2.source.value_counts())

Twitter for iPhone              631
Twitter for Android             225
Twitter Web App                 72 
Instagram                       40 
Twitter Web Client              25 
Twitter for iPad                18 
Whats The Wait?                 13 
twittbot.net                    11 
DisneyCStats                    9  
TdrStats                        6  
Poplle(ポップル)-いいね!でお金がもらえるSNS    6  
realtimedisneytweet             6  
Poshmark                        5  
IFTTT                           5  
TweetDeck                       4  
dlvr.it                         4  
Foursquare                      4  
Mice Town                       3  
feather for iOS                 2  
Peing                           2  
Disney Real                     2  
Foursquare Swarm                2  
WordPress.com                   2  
starwars app                    2  
Botbird tweets                  2  
Facebook                        2  
twicca                          1  
Vaccines and Homeopathy News

In [47]:
#look at which columns are useful and eliminite unnecessary columns 
#row 0 all columns 
df2.loc[0,:]
#multiple rows
df2.loc[[0,1,2],:]
#or will include the 2 inclusive on both sides 
df2.loc[0:2,:]
#all rows text and language
df2.loc[0:2,['text','lang']]
#set all rows that all rows are ja(japanese) then column text
df2.loc[df2.lang == 'ja','text']

#Iloc filter rows and select columns by integer position 
df2.iloc[:,0:4] #with iloc it is exclusive of the second of the second number inclusive of the first number

#want 0 1 and 2 and all columns
df2.iloc[0:2, :]
#ix figure out position or label
df2.ix[0:2,'text':'source']

/Users/justin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,text,id,lang,created_at,source
0,RT @tdrblognote: これが新しいミッキー花壇ですか～ 分離型です。イベントフォトロケを作るなら、ワールドバザール側ですかね。ミッキー側はフォトロケ置けるスペースが無い。 #TDR_now https://t.co/xYEQmaDLHm,1176336509575712768,ja,2019-09-24 03:24:13,Twitter for iPhone
1,"@OrangeGrove55 Galaxy’s Edge at Walt Disney World isn’t a failure, but like Disneyland it’s underperforming. But ho… https://t.co/3KdOXXhmNV",1176336519969345536,en,2019-09-24 03:24:16,Twitter for iPhone
2,RT @hp_yec: 昨日となんか違うと思ったら\nミッキー花壇帰ってました！\nお帰り！\n\n#TDR_now https://t.co/KUib5PtJlT,1176336531339919360,ja,2019-09-24 03:24:19,Twitter for iPhone


In [33]:
df2.shape

(1118, 20)

In [49]:
df2.iloc[0:2,:]

text  \
0  RT @tdrblognote: これが新しいミッキー花壇ですか～ 分離型です。イベントフォトロケを作るなら、ワールドバザール側ですかね。ミッキー側はフォトロケ置けるスペースが無い。 #TDR_now https://t.co/xYEQmaDLHm                   
1  @OrangeGrove55 Galaxy’s Edge at Walt Disney World isn’t a failure, but like Disneyland it’s underperforming. But ho… https://t.co/3KdOXXhmNV   

                    id lang          created_at              source  \
0  1176336509575712768  ja  2019-09-24 03:24:13  Twitter for iPhone   
1  1176336519969345536  en  2019-09-24 03:24:16  Twitter for iPhone   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [57]:
#textblob translation 
# from textblob import TextBlob
# blob = TextBlob('これが新しいミッキー花壇ですか～ 分離型です。イベントフォトロケを作るなら、ワールドバザール側ですかね。ミッキー側はフォトロケ置けるスペースが無い。 #TDR_now https://t.co/xYEQmaDLHm')
# blob.translate('en')

NotTranslated: Translation API returned the input string unchanged.

In [102]:
from googletrans import Translator
translator = Translator()
print(translator.translate('これが新しいミッキー花壇ですか～ 分離型です。イベントフォトロケを作るなら、ワールドバザール側ですかね。ミッキー側はフォトロケ置けるスペースが無い'))

Translated(src=ja, dest=en, text=This is the new Mickey flower beds Is-separation type. If you create an event photo location, I guess is the World Bazaar side. Mickey side is not Fotoroke put space, pronunciation=None, extra_data="{'translat...")


#sentiment analysis 

http://makemeanalyst.com/sentiment-analysis-products-for-english-spanish-and-japanese/

#another method to clean json data 

In [103]:
import json 

In [106]:
# with open('disresort1.json') as access_json:
#     read_content = json.load(access_json)